# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff020b218b0>
├── 'a' --> tensor([[-1.6390, -0.7130, -0.6228],
│                   [-0.1865,  1.7135, -1.4252]])
└── 'x' --> <FastTreeValue 0x7ff020b217f0>
    └── 'c' --> tensor([[-1.7669, -0.9698,  0.7211,  0.6404],
                        [ 0.7578,  0.4488, -0.6000,  0.3691],
                        [ 0.2290,  0.7236, -0.3947, -0.2187]])

In [4]:
t.a

tensor([[-1.6390, -0.7130, -0.6228],
        [-0.1865,  1.7135, -1.4252]])

In [5]:
%timeit t.a

66.6 ns ± 0.0233 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff020b218b0>
├── 'a' --> tensor([[-0.1546,  1.6427, -1.7318],
│                   [ 0.0184, -0.4183, -0.3194]])
└── 'x' --> <FastTreeValue 0x7ff020b217f0>
    └── 'c' --> tensor([[-1.7669, -0.9698,  0.7211,  0.6404],
                        [ 0.7578,  0.4488, -0.6000,  0.3691],
                        [ 0.2290,  0.7236, -0.3947, -0.2187]])

In [7]:
%timeit t.a = new_value

60.6 ns ± 0.0523 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.6390, -0.7130, -0.6228],
               [-0.1865,  1.7135, -1.4252]]),
    x: Batch(
           c: tensor([[-1.7669, -0.9698,  0.7211,  0.6404],
                      [ 0.7578,  0.4488, -0.6000,  0.3691],
                      [ 0.2290,  0.7236, -0.3947, -0.2187]]),
       ),
)

In [10]:
b.a

tensor([[-1.6390, -0.7130, -0.6228],
        [-0.1865,  1.7135, -1.4252]])

In [11]:
%timeit b.a

53.8 ns ± 0.0353 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.1776, -1.5274, -1.2549],
               [-0.1396, -0.4943, -0.6728]]),
    x: Batch(
           c: tensor([[-1.7669, -0.9698,  0.7211,  0.6404],
                      [ 0.7578,  0.4488, -0.6000,  0.3691],
                      [ 0.2290,  0.7236, -0.3947, -0.2187]]),
       ),
)

In [13]:
%timeit b.a = new_value

481 ns ± 0.112 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

984 ns ± 17.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 24 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

139 µs ± 220 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 133 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff020b18bb0>
├── 'a' --> tensor([[[-1.6390, -0.7130, -0.6228],
│                    [-0.1865,  1.7135, -1.4252]],
│           
│                   [[-1.6390, -0.7130, -0.6228],
│                    [-0.1865,  1.7135, -1.4252]],
│           
│                   [[-1.6390, -0.7130, -0.6228],
│                    [-0.1865,  1.7135, -1.4252]],
│           
│                   [[-1.6390, -0.7130, -0.6228],
│                    [-0.1865,  1.7135, -1.4252]],
│           
│                   [[-1.6390, -0.7130, -0.6228],
│                    [-0.1865,  1.7135, -1.4252]],
│           
│                   [[-1.6390, -0.7130, -0.6228],
│                    [-0.1865,  1.7135, -1.4252]],
│           
│                   [[-1.6390, -0.7130, -0.6228],
│                    [-0.1865,  1.7135, -1.4252]],
│           
│                   [[-1.6390, -0.7130, -0.6228],
│                    [-0.1865,  1.7135, -1.4252]]])
└── 'x' --> <FastTreeValue 0x7ff020b18c40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.2 µs ± 81.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff020ae0280>
├── 'a' --> tensor([[-1.6390, -0.7130, -0.6228],
│                   [-0.1865,  1.7135, -1.4252],
│                   [-1.6390, -0.7130, -0.6228],
│                   [-0.1865,  1.7135, -1.4252],
│                   [-1.6390, -0.7130, -0.6228],
│                   [-0.1865,  1.7135, -1.4252],
│                   [-1.6390, -0.7130, -0.6228],
│                   [-0.1865,  1.7135, -1.4252],
│                   [-1.6390, -0.7130, -0.6228],
│                   [-0.1865,  1.7135, -1.4252],
│                   [-1.6390, -0.7130, -0.6228],
│                   [-0.1865,  1.7135, -1.4252],
│                   [-1.6390, -0.7130, -0.6228],
│                   [-0.1865,  1.7135, -1.4252],
│                   [-1.6390, -0.7130, -0.6228],
│                   [-0.1865,  1.7135, -1.4252]])
└── 'x' --> <FastTreeValue 0x7feff80372b0>
    └── 'c' --> tensor([[-1.7669, -0.9698,  0.7211,  0.6404],
                        [ 0.7578,  0.4488, -0.6000,  0.3691],
                 

In [23]:
%timeit t_cat(trees)

30.3 µs ± 43.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

60.5 µs ± 165 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.6390, -0.7130, -0.6228],
                [-0.1865,  1.7135, -1.4252]],
       
               [[-1.6390, -0.7130, -0.6228],
                [-0.1865,  1.7135, -1.4252]],
       
               [[-1.6390, -0.7130, -0.6228],
                [-0.1865,  1.7135, -1.4252]],
       
               [[-1.6390, -0.7130, -0.6228],
                [-0.1865,  1.7135, -1.4252]],
       
               [[-1.6390, -0.7130, -0.6228],
                [-0.1865,  1.7135, -1.4252]],
       
               [[-1.6390, -0.7130, -0.6228],
                [-0.1865,  1.7135, -1.4252]],
       
               [[-1.6390, -0.7130, -0.6228],
                [-0.1865,  1.7135, -1.4252]],
       
               [[-1.6390, -0.7130, -0.6228],
                [-0.1865,  1.7135, -1.4252]]]),
    x: Batch(
           c: tensor([[[-1.7669, -0.9698,  0.7211,  0.6404],
                       [ 0.7578,  0.4488, -0.6000,  0.3691],
                       [ 0.2290,  0.7236, -0.3947, -0.2187]],
         

In [26]:
%timeit Batch.stack(batches)

79.3 µs ± 132 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.6390, -0.7130, -0.6228],
               [-0.1865,  1.7135, -1.4252],
               [-1.6390, -0.7130, -0.6228],
               [-0.1865,  1.7135, -1.4252],
               [-1.6390, -0.7130, -0.6228],
               [-0.1865,  1.7135, -1.4252],
               [-1.6390, -0.7130, -0.6228],
               [-0.1865,  1.7135, -1.4252],
               [-1.6390, -0.7130, -0.6228],
               [-0.1865,  1.7135, -1.4252],
               [-1.6390, -0.7130, -0.6228],
               [-0.1865,  1.7135, -1.4252],
               [-1.6390, -0.7130, -0.6228],
               [-0.1865,  1.7135, -1.4252],
               [-1.6390, -0.7130, -0.6228],
               [-0.1865,  1.7135, -1.4252]]),
    x: Batch(
           c: tensor([[-1.7669, -0.9698,  0.7211,  0.6404],
                      [ 0.7578,  0.4488, -0.6000,  0.3691],
                      [ 0.2290,  0.7236, -0.3947, -0.2187],
                      [-1.7669, -0.9698,  0.7211,  0.6404],
                      [ 0.7578,  

In [28]:
%timeit Batch.cat(batches)

143 µs ± 282 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

307 µs ± 2.37 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
